In [1]:
from utils import r14, r32, r50, r125, r134a, r143a, r170, atom_type, opt_atom_types
import numpy as np
import unyt as u
import pandas as pd
from fffit.fffit.utils import values_real_to_scaled, values_scaled_to_real, variances_scaled_to_real
from fffit.fffit.plot import plot_obj_contour

2024-04-16 18:29:24.782283: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 18:29:24.826619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 18:29:24.826665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 18:29:24.828093: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-16 18:29:24.835408: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 18:29:24.836011: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
#Load class properies for each molecule
r14_class = r14.R14Constants()
r32_class = r32.R32Constants()
r50_class = r50.R50Constants()
r125_class = r125.R125Constants()
r134a_class = r134a.R134aConstants()
r143a_class = r143a.R143aConstants()
r170_class = r170.R170Constants()

In [5]:
at_class = atom_type.AT_Scheme_9()
w_calc = 2
repeats = 1
seed = 1
save_data = False
w_calc = 2
obj_choice = "ExpVal"
molec_data_dict = {"R134a":r134a_class}
pm = at_class.get_transformation_matrix(molec_data_dict)
# print(pm)
theta_paper = np.array([3.727, 3.578558657, 3.754, 2.237, 2.607, 3.255, 2.982, 71.73, 11.25885226, 72.61, 23.13, 39.98, 20.164, 23.13])
t_paper = theta_paper.reshape(-1,1).T@pm
# print(t_paper)

In [22]:
import scipy 
repeats = 1
seed = 1
save_data = False
w_calc = 2
obj_choice = "ExpVal"
at_class = atom_type.AT_Scheme_10()
# molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R143a":r143a_class}
molec_data_dict = {"R134a":r134a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
at_optimizer =  opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, w_calc, obj_choice, save_data)
# best_pref = np.array([3.727, 3.578558657, 3.754, 2.237, 2.607, 3.255, 2.982, 71.73, 11.25885226, 72.61, 2.55, 39.98, 20.164, 23.13])
best_pref = np.array([3.727,3.810,3.754, 3.028,3.809,
                      2.316,
                      2.607,3.255,3.424,2.917,
                      71.730,52.200, 72.610,53.690,69.970,
                      6.160,
                      39.980,20.164, 17.050, 29.070])
best_real= at_optimizer.values_pref_to_real(best_pref)
obj = at_optimizer.calc_obj(best_real)
print(obj[0])

# jac, hess = at_optimizer.approx_jac_hess(best_real)
# print(hess.shape)
# print(scipy.linalg.norm(jac, ord = np.inf))
# eigval, eigvec = scipy.linalg.eig(hess)
# print(eigval, eigvec)

152401.03174074026


In [ ]:
# best_pref = np.array([3.727, 3.037, 3.809, 1.711, 2.607, 3.255, 3.424, 71.73, 55.4, 69.97, 3.55, 39.98, 20.164, 17.05])
print(best_pref)
#Get transformation matrix
param_matrix = at_class.get_transformation_matrix(molec_data_dict)
#Transform the guess, and scale to bounds
best_real= at_optimizer.values_pref_to_real(best_pref)
print(best_real)
gp_theta = best_real.reshape(-1,1).T@param_matrix
print(gp_theta)
gp_theta_guess = values_real_to_scaled(gp_theta, molec_object.param_bounds)
print(gp_theta_guess)

molec_gps_dict = all_gp_dict[molec_key]
key = list(molec_gps_dict.keys())[0]
#Get GP associated with property
gp_model = molec_gps_dict[key]
#Get X and Y data and bounds associated with the GP
exp_data, y_bounds, y_names = at_optimizer.get_exp_data(molec_object, key)
#Get x and y data
x_exp = np.array(list(exp_data.keys())).reshape(-1,1)
y_exp = np.array(list(exp_data.values()))

gp_mean_scl, gp_covar_scl, gp_var_scl, gp_grad_mean = at_optimizer.eval_gp_new_theta(gp_theta_guess, molec_object, gp_model, x_exp)
#Scale gp output to real value
gp_mean = values_scaled_to_real(gp_mean_scl, y_bounds)
print(gp_mean.flatten() - y_exp.flatten())

# jac, hess = at_optimizer.approx_jac_hess(best_real)
# print(scipy.linalg.norm(jac, ord = np.inf))

In [19]:
import os
import tensorflow as tf
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
visual = opt_atom_types.Vis_Results(molec_data_dict, all_gp_dict, at_class, w_calc, obj_choice, save_data)
for molec_names in list(molec_data_dict.keys()):
    param_matrix = visual.at_class.get_transformation_matrix({molec_names: molec_data_dict[molec_names]})
    #Get best_per_run.csv for all molecules
    molec_dir = visual.make_results_dir([molec_names])
    if os.path.exists(molec_dir+"/best_per_run.csv"):
        print(molec_dir)
        molec_df = pd.read_csv(molec_dir+"/best_per_run.csv", header = 0)
        # print(molec_df.head())
        first_param_name = at_class.at_names[0] + "_min"
        last_param_name = at_class.at_names[-1] + "_min"
        print(molec_df.loc[0, "Min Obj"])
        molec_best = molec_df.loc[0, first_param_name:last_param_name].values
        print(molec_best)
        # ind_best_real = visual.values_pref_to_real(molec_best)
        # ind_best_scl = values_real_to_scaled(ind_best_real.reshape(1,-1), at_class.at_bounds_nm_kjmol)
        ind_best_gp = molec_best.reshape(-1,1).T@param_matrix
        # ind_best_gp = tf.convert_to_tensor(ind_best_gp, dtype=tf.float64)
        print(ind_best_gp)

Results/at_10/R14/ExpVal/wt_y_var
661.8078906881692
[3.436460946865524 3.716627672858084 3.51016437693536 3.3961144968946053
 3.7289588601092 1.98402149525512 3.3415775815751743 2.9017478728266983
 2.7642055040630344 2.9198839801773446 30.47567970449768
 30.629956487288005 50.424758914819606 37.96607260338219 73.2971350652714
 7.42240713147449 21.94994609494888 29.93453532718115 27.017118392104805
 32.3689403554371]
[[3.436460946865524 2.9198839801773446 30.47567970449768 32.3689403554371]]
Results/at_10/R32/ExpVal/wt_y_var
8.084276704417292
[3.810436510734817 3.054116204515218 3.771884198621549 2.7145395200049998
 3.817070301839598 1.9857937949104725 2.031642485693112 2.706935603559554
 3.3817938350338475 3.994645700902961 48.7342408397191 18.91382372587706
 70.61870509741566 55.29318049684352 14.290011226934064 7.627433223174589
 41.38566659614363 38.73236949180481 39.90336655199651 19.34948366902577]
[[3.810436510734817 2.706935603559554 1.9857937949104725 48.7342408397191
  38.7323

In [ ]:
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
df = pd.DataFrame(columns = ["Molecule", "Property", "Value", "Uncertainty", "Weight"])
for key in list(molec_data_dict.keys()):
    molec_object = molec_data_dict[key]
    for prop_key in list(molec_object.uncertainties.keys()):
        if "vap_density" in prop_key:
            exp_data = molec_object.expt_vap_density
            property_bounds = molec_object.vap_density_bounds
            property_name = "Vapor Density [kg/m^3]"
        elif "liq_density" in prop_key:
            exp_data = molec_object.expt_liq_density
            property_bounds = molec_object.liq_density_bounds
            property_name = "Liquid Density [kg/m^3]"
        elif "Pvap" in prop_key: 
            exp_data = molec_object.expt_Pvap
            property_bounds = molec_object.Pvap_bounds
            property_name = "Vapor pressure [bar]"
        elif "Hvap" in prop_key:
            exp_data = molec_object.expt_Hvap
            property_bounds = molec_object.Hvap_bounds
            property_name = "Enthalpy of Vaporization [kJ/kg]"

        values = np.array(list(exp_data.values()))
        y_unc = molec_object.uncertainties[prop_key]
        weight_unc = 1/(y_unc*values)**2
        weight_2pct = 1/(0.02*values)**2
        weight = np.maximum(weight_unc, weight_2pct)
        df_vals = [key, prop_key, values, y_unc, weight]
        iter_df = pd.DataFrame([df_vals], columns = df.columns)
        iter_df = iter_df.apply(lambda col: col.explode(), axis=0).reset_index(drop=True).copy(deep =True)
        df = pd.concat([df, iter_df], ignore_index=True)
# df.to_csv("molecule_exp_unc_data.csv", index = False, header = True)

In [ ]:
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
molec_data_dict = {"R125":r125_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
visual = opt_atom_types.Vis_Results(molec_data_dict, all_gp_dict, at_class, w_calc, obj_choice, save_data)
molec_names = list(["R14", "R32", "R50", "R170", "R125", "R134a", "R143a"])
visual.comp_paper_full_ind(molec_names)

In [ ]:
molec = "R32"
molec_data_dict = {molec:r32_class} #, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
# all_gp_dict = {"R14" : {list(all_gp_dict["R14"].keys())[0]: list(all_gp_dict["R14"].values())[0]}}
# all_gp_dict = {molec : all_gp_dict[molec]}
at_optimizer =  opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, w_calc, obj_choice, save_data)
ls_results = at_optimizer.optimize_ats()
ls_results

In [ ]:
# Determine the index of the row where column 'B' has the minimum value
min_index = ls_results['Min Obj Cum.'].idxmin()

# Retrieve the row corresponding to the minimum value in column 'B'
min_row = ls_results.iloc[min_index]
print(min_row)

In [ ]:
first_param_name = at_optimizer.at_class.at_names[0] + "_min"
last_param_name = at_optimizer.at_class.at_names[-1] + "_min"
molec_best = ls_results.loc[21, first_param_name:last_param_name].values
print(molec_best)

In [ ]:
#Get best values from csvs
def get_best_results(visual, molec_names):
    best_vals = {}
    #Get best_per_run.csv for all molecules
    all_molec_dir = visual.make_results_dir(molec_names)
    all_df = pd.read_csv(all_molec_dir+"/best_per_run.csv", header = 0)
    # print(all_df)
    first_param_name = visual.at_class.at_names[0] + "_min"
    last_param_name = visual.at_class.at_names[-1] + "_min"
    full_opt_best = all_df.loc[0, first_param_name:last_param_name].values
    best_vals["All"] = full_opt_best
    for i in range(len(molec_names)):
        molec_dir = visual.make_results_dir([molec_names[i]])
        molec_df = pd.read_csv(molec_dir+"/best_per_run.csv", header = 0)
        molec_best = molec_df.loc[0, first_param_name:last_param_name].values
        best_vals[molec_names[i]] = molec_best
    return best_vals